### Colegio Anglo Americano 

#### Transacciones PDF 

In [1]:
pip install pymupdf pdfplumber tabula-py camelot-py[cv] pytesseract Pillow pandas camelot

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Lectura de pdf
import fitz  
import pdfplumber
import tabula
import pytesseract
from PIL import Image
import pandas as pd
import io
import re
import pandas as pd
from datetime import datetime

In [3]:


class PDFExtractorAdaptativo:
    def __init__(self):
        self.metodos_texto = ['pdfplumber', 'pymupdf', 'ocr']
        self.metodos_tablas = ['tabula', 'pdfplumber']  # Quitamos 'camelot'
    
    def detectar_tipo_contenido(self, archivo_pdf):
        """Detecta qué tipo de contenido tiene el PDF"""
        try:
            doc = fitz.open(archivo_pdf)
            primera_pagina = doc[0]
            
            texto = primera_pagina.get_text().strip()
            imagenes = primera_pagina.get_images()
            
            with pdfplumber.open(archivo_pdf) as pdf:
                page = pdf.pages[0]
                tablas_detectadas = page.find_tables()
            
            resultado = {
                'tiene_texto': len(texto) > 50,
                'tiene_imagenes': len(imagenes) > 0,
                'posibles_tablas': len(tablas_detectadas) > 0,
                'es_escaneado': len(texto) < 50 and len(imagenes) > 0,
                'calidad_texto': 'alta' if len(texto) > 200 else 'baja'
            }
            
            doc.close()
            return resultado
            
        except Exception as e:
            return {'error': str(e)}
    
    def extraer_texto_robusto(self, archivo_pdf):
        """Extrae texto probando diferentes métodos"""
        metodos_resultados = {}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                texto = ""
                for page in pdf.pages:
                    texto += page.extract_text() + "\n"
                metodos_resultados['pdfplumber'] = texto
        except Exception as e:
            metodos_resultados['pdfplumber'] = f"Error: {e}"
        
        # pymupdf
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page in doc:
                texto += page.get_text() + "\n"
            doc.close()
            metodos_resultados['pymupdf'] = texto
        except Exception as e:
            metodos_resultados['pymupdf'] = f"Error: {e}"
        
        # OCR
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                pix = page.get_pixmap()
                img_data = pix.tobytes("png")
                img = Image.open(io.BytesIO(img_data))
                texto += pytesseract.image_to_string(img, lang='spa') + "\n"
            doc.close()
            metodos_resultados['ocr'] = texto
        except Exception as e:
            metodos_resultados['ocr'] = f"Error: {e}"
        
        # Seleccionar mejor resultado
        mejor_resultado = ""
        max_longitud = 0
        for metodo, resultado in metodos_resultados.items():
            if not resultado.startswith("Error") and len(resultado) > max_longitud:
                mejor_resultado = resultado
                max_longitud = len(resultado)
        
        return {
            'texto_final': mejor_resultado,
            'metodos_intentados': metodos_resultados,
            'metodo_exitoso': max([k for k, v in metodos_resultados.items() 
                                 if not v.startswith("Error") and len(v) == max_longitud], 
                                default="ninguno")
        }
    
    def extraer_tablas_robusto(self, archivo_pdf):
        """Extrae tablas probando diferentes métodos (sin camelot)"""
        resultados_tablas = {}
        
        # tabula-py
        try:
            tablas = tabula.read_pdf(archivo_pdf, pages='all', multiple_tables=True)
            resultados_tablas['tabula'] = {
                'tablas': tablas,
                'cantidad': len(tablas),
                'exito': True
            }
        except Exception as e:
            resultados_tablas['tabula'] = {'error': str(e), 'exito': False}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                todas_tablas = []
                for page in pdf.pages:
                    tablas_pagina = page.extract_tables()
                    for tabla in tablas_pagina:
                        df = pd.DataFrame(tabla[1:], columns=tabla[0])
                        todas_tablas.append(df)
                
                resultados_tablas['pdfplumber'] = {
                    'tablas': todas_tablas,
                    'cantidad': len(todas_tablas),
                    'exito': True
                }
        except Exception as e:
            resultados_tablas['pdfplumber'] = {'error': str(e), 'exito': False}
        
        return resultados_tablas
    
    def procesar_pdf_completo(self, archivo_pdf):
        """Proceso completo adaptativo"""
        print(f"Analizando: {archivo_pdf}")
        
        info_contenido = self.detectar_tipo_contenido(archivo_pdf)
        print(f"Análisis de contenido: {info_contenido}")
        
        resultado_final = {
            'archivo': archivo_pdf,
            'analisis_contenido': info_contenido,
            'texto': None,
            'tablas': None
        }
        
        if info_contenido.get('tiene_texto') or info_contenido.get('es_escaneado'):
            print("Extrayendo texto...")
            resultado_texto = self.extraer_texto_robusto(archivo_pdf)
            resultado_final['texto'] = resultado_texto
        
        if info_contenido.get('posibles_tablas'):
            print("Extrayendo tablas...")
            resultado_tablas = self.extraer_tablas_robusto(archivo_pdf)
            resultado_final['tablas'] = resultado_tablas
        
        return resultado_final

# Ejemplo de uso
if __name__ == "__main__":
    extractor = PDFExtractorAdaptativo()
    resultado = extractor.procesar_pdf_completo("resources/COLEGIO_ANGLO_AMERICANO.pdf")


Analizando: resources/COLEGIO_ANGLO_AMERICANO.pdf
Análisis de contenido: {'tiene_texto': True, 'tiene_imagenes': True, 'posibles_tablas': True, 'es_escaneado': False, 'calidad_texto': 'alta'}
Extrayendo texto...


Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


Extrayendo tablas...


In [4]:
#funcion de limpieza de columnas : 
def limpiar_columnas(df):
    df.columns = (
        df.columns
        .str.strip()                      
        .str.lower()                     
        .str.replace(' ', '_')        
        .str.replace(r'[^\w\s]', '', regex=True)  
    )
    return df

In [5]:
#extraer del diccionario final tablas y texto
textos = resultado.get('texto', {}).get('texto_final', '')
print(type(textos))

#Escritura .txt
with open("COLEGIO_ANGLO_AMERICANO.txt", "w", encoding="utf-8") as f:
    f.write(textos)

<class 'str'>


In [ ]:
#Regex
data = {}

# Regex para datos principales
data['cliente'] = re.search(r'Fundación Colegio.*', textos).group(0).strip()
data['nit_cliente'] = re.search(r'NIT:\s*([\d\.-]+)', textos).group(1)
data['fecha'] = re.search(r'\b\d{2}/\d{2}/\d{4}\b', textos).group(0)
data['autorizado_por'] = re.search(r'Autorizado por:\s*(.+)', textos).group(1).strip()
data['proveedor'] = re.search(r'Proveedor:\s*(.+)', textos).group(1).strip()
data['direccion_proveedor'] = re.search(r'Proveedor:\s*.+\n(.+)', textos).group(1).strip()
data['nit_proveedor'] = re.search(r'\n(\d{9})\n', textos).group(1)
data['condicion_pago'] = re.search(r'Condicion de Pago:\s*(.+)', textos).group(1).strip()

In [28]:
data

{'cliente': 'Fundación Colegio Anglo Colombiano',
 'nit_cliente': '860.009.107-0',
 'fecha': '08/05/2025',
 'autorizado_por': 'Constanza Sarmiento Fernández',
 'proveedor': 'Dirección:',
 'direccion_proveedor': 'Teléfono:',
 'telefono_proveedor': '0025500',
 'nit_proveedor': '830125610',
 'condicion_pago': 'Proveedor:'}

In [32]:
#Regex de Productos 
producto_match = re.search(r'CHICHARRON.*?LIBRA', textos)
if producto_match:
    data['cantidad'] = re.search(r'\$\s+([\d,\.]+)', textos).group(1).replace(',', '')
    data['subtotal'] = re.findall(r'\n\s*([\d,\.]+)\n\$', textos)[0].replace(',', '')
    data['total'] = re.findall(r'\n\s*([\d,\.]+)\nYEISON', textos)[0].replace(',', '')
    data['iva'] = re.findall(r'\n\s*([\d,\.]+)\n\$', textos)[1].replace(',', '')


In [33]:
df = pd.DataFrame([data])

In [34]:
df

,cliente,nit_cliente,fecha,autorizado_por,proveedor,direccion_proveedor,telefono_proveedor,nit_proveedor,condicion_pago,cantidad,valor_unitario,subtotal,total,iva
0,Fundación Colegio Anglo Colombiano,860.009.107-0,08/05/2025,Constanza Sarmiento Fernández,Dirección:,Teléfono:,0025500,830125610,Proveedor:,20.00,20.00,773222,777000.00,0.00


#### Escoger Columnas Productos 
1. Numero de item
2. descripcion
3. Cantidad
4. Unidad de medida
5. Fecha de entrega
6. Costo Unitario
7. importe
8. id falso (Unir para realizar insersion de datos)

In [16]:
textos

'F. ENTREGA\nTOTAL\nIVA\nDSCTO\nVLR UND.\nMEDIDA\nDESCRIPCION\nCANT.\nREF.\n0025500\nFundación Colegio Anglo Colombiano\nNIT: 860.009.107-0\nBogotá, Colombia, Sur América\nAv. 19 No. 152A -48\nPBX :(571) 259 57 00\nFecha:\nAutorizado por:\nConstanza Sarmiento Fernández\nCotización No.\nSolicitud No.\nCondicion de Pago:\nProveedor:\nDirección:\nTeléfono:\nCiudad:\nNit:\nCROC S.A.S\nCL 69 19 36\n5448274\n830125610\n08/05/2025\n0017686\n30 DÍAS\n0021449\nORDEN DE COMPRA\nPagina\nFecha de aprobación:\nPágina 1 de 1\nBOGOTÁ\nICUI\n773222\n$\n 20.00\n 740,000.00\nLIBRA\n10/05/2025\n$  37,000.00\n 0.00 %\n 5\n%\nCHICHARRON CARNUDO -  - LIBRA \n-UND \n 0 %\nTotal bruto\nTOTAL \nVALOR:\nSubtotal\nDescuento\nIVA\nAutorizado por:\nNOTA:  Para entregar el bien o servicio adjuntar copia de este documento\nLa información contenida en este documento es para uso interno de la Fundación Colegio Anglo Colombiano y es de uso confidencial\nCualquier copia impresa de este documento sera una copia NO CONTRO

### Escoger Columnas -Ordenar dataframe - Dataframe General 
1. Cliente
2. Direccion Cliente
3. Destino
4. Numero de Pedido
5. Fecha de Orden
6. Fecha de Entrega
7. Id Falso (Para Cruce de Orden de compra )
